In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
import os

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from torch.nn.functional import softmax

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm


In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.1.1+cu121
True
NVIDIA RTX A4000


In [3]:
cpu_cores = os.cpu_count()
print("Number of CPU cores available:", cpu_cores)

Number of CPU cores available: 8


In [4]:
data_dir = "/storage/pap_vs_normal"

In [5]:
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)

n_total = len(full_dataset)
n_train = int(0.7 * n_total)
n_val   = int(0.15 * n_total)
n_test  = n_total - n_train - n_val

train_ds, val_ds, test_ds = random_split(full_dataset, [n_train, n_val, n_test])

train_ds.dataset.transform = train_transform
val_ds.dataset.transform = base_transform
test_ds.dataset.transform = base_transform

In [8]:
batch_size = 8

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PapVsNormalCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 3 × 224 × 224 input

        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),      # 112 × 112

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),      # 56 × 56

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),      # 28 × 28

            # Block 4 (more abstract features)
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),      # 14 × 14
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),   # 256 × 1 × 1
            nn.Flatten(),              # 256
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, 3)          # binary logit
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = PapVsNormalCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


In [19]:
def run_epoch(loader, train=True):
    model.train() if train else model.eval()

    total_loss, correct, total = 0.0, 0, 0

    for xb, yb in loader:
        xb = xb.to(device)
        yb = yb.to(device).long()  # [B]

        with torch.set_grad_enabled(train):
            logits = model(xb)          # [B,3]
            loss = criterion(logits, yb)

            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * xb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    return total_loss / total, correct / total
num_epochs = 30

for epoch in range(num_epochs):
    train_loss, train_acc = run_epoch(train_loader, train=True)
    val_loss,   val_acc   = run_epoch(val_loader,   train=False)

    print(
        f"Epoch {epoch+1:02d} | "
        f"train_loss={train_loss:.4f} acc={train_acc:.3f} | "
        f"val_loss={val_loss:.4f} acc={val_acc:.3f}"
    )

Epoch 01 | train_loss=0.7340 acc=0.706 | val_loss=0.5677 acc=0.798
Epoch 02 | train_loss=0.6164 acc=0.763 | val_loss=0.4919 acc=0.798
Epoch 03 | train_loss=0.5466 acc=0.784 | val_loss=0.4089 acc=0.853
Epoch 04 | train_loss=0.5104 acc=0.803 | val_loss=0.4733 acc=0.802
Epoch 05 | train_loss=0.4524 acc=0.824 | val_loss=0.3513 acc=0.880
Epoch 06 | train_loss=0.4559 acc=0.837 | val_loss=0.3925 acc=0.872
Epoch 07 | train_loss=0.4015 acc=0.866 | val_loss=0.3001 acc=0.888
Epoch 08 | train_loss=0.3662 acc=0.867 | val_loss=0.2795 acc=0.903
Epoch 09 | train_loss=0.3612 acc=0.871 | val_loss=0.3097 acc=0.864
Epoch 10 | train_loss=0.3840 acc=0.865 | val_loss=0.2702 acc=0.915
Epoch 11 | train_loss=0.3127 acc=0.887 | val_loss=0.2659 acc=0.899
Epoch 12 | train_loss=0.2968 acc=0.896 | val_loss=0.3309 acc=0.880
Epoch 13 | train_loss=0.2937 acc=0.900 | val_loss=0.2400 acc=0.907
Epoch 14 | train_loss=0.2910 acc=0.901 | val_loss=0.2243 acc=0.911
Epoch 15 | train_loss=0.2415 acc=0.925 | val_loss=0.2680 acc=0

In [20]:
model.eval()
test_loss, test_acc = run_epoch(test_loader, train=False)
print(f"TEST | loss={test_loss:.4f} acc={test_acc:.3f}")


TEST | loss=0.1834 acc=0.923
